In [33]:
import spacy
import nltk
from pprint import pprint
import random
from sklearn import preprocessing
import numpy as np
nltk.download("all")

nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/miles/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/miles/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]   

In [34]:
import os
def load_corpus(folder):
    corpus = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            try:
                with open(os.path.join(root, name), "r") as fp:
                    corpus.append(fp.read())
            except UnicodeDecodeError as e:
                print(e.__str__(), "for", os.path.join(root, name))
    return corpus

def load_corpuses(folder):
    sub_folders = []
    for root, dirs, files in os.walk(folder):
        if dirs:
            for dir_ in dirs:
                sub_folders.append(dir_)

    corpuses = {}
    print(sub_folders)
    for sub_folder in sub_folders:
        print(os.path.join(folder, sub_folder))
        corpuses[sub_folder] = load_corpus(os.path.join(folder, sub_folder))
    return corpuses

corpuses = load_corpuses("bbc")

['business', 'tech', 'entertainment', 'sport', 'politics']
bbc/business
bbc/tech
bbc/entertainment
bbc/sport
'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte for bbc/sport/199.txt
bbc/politics


In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
tfid = TfidfTransformer()

all_ = []
for corpus in corpuses:
    all_ += corpuses[corpus]

vectorizer.fit(all_)
tfid.fit(vectorizer.transform(all_))

# x = vectorizer.transform(corpuses["tech"])
# vectorizer.vocabulary_.get('Sony')

TfidfTransformer()

In [36]:
vectorizer.transform(['regulatory approval']).toarray()[0]


array([0, 0, 0, ..., 0, 0, 0])

In [37]:
import math
x = []
y =[]
from tqdm.notebook import tqdm

entity_types = CountVectorizer(stop_words=stopwords.words('english'))
entity_types.fit(['CARDINAL', 'PERSON', 'GPE', 'MONEY', 'ORG', 'ORDINAL', 'WORK_OF_ART', 'NORP', 'PERCENT', 'DATE', 'LANGUAGE', 'FAC', 'LOC', 'TIME', 'PRODUCT', 'EVENT', 'QUANTITY', 'LAW'])
for corpus in tqdm(corpuses):
    for story in tqdm(corpuses[corpus]):
        analysed = nlp(story)
        print(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0])
        print(vectorizer.transform([story]).toarray()[0])
        x.append(list(vectorizer.transform([story]).toarray()[0]) + list(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0]))
        # x.append(tfid.transform(vectorizer.transform([story])).toarray()[0])

        y.append(corpus)
pprint(x[:3])
pprint(y[:3])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/510 [00:00<?, ?it/s]

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 7 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 2 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 3 0 ...

  0%|          | 0/401 [00:00<?, ?it/s]

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 2 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ...

...

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ...

  0%|          | 0/510 [00:00<?, ?it/s]

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ...

  0%|          | 0/417 [00:00<?, ?it/s]

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 2 0 ... 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 ...

In [46]:
size_dataset_full=len(x)
size_test=int(round(size_dataset_full*0.2,0))

list_test_indices=random.sample(range(size_dataset_full), size_test)

test_x = []
test_y = []
train_x = []
train_y = []

for i,example in enumerate(x):
  if i in list_test_indices:
      test_x.append(example)
      test_y.append(y[i])
  else:
      train_x.append(example)
      train_y.append(y[i])

le = preprocessing.LabelEncoder()
le.fit(train_y + test_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

In [48]:
print(test_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2]


In [58]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

svm_clf=make_pipeline(StandardScaler(), svm.SVC(cache_size=2000, decision_function_shape='ovo'))

In [59]:
svm_clf.fit(train_x, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(cache_size=2000, decision_function_shape='ovo'))])

In [60]:
Y_text_predictions = svm_clf.predict(test_x)

In [61]:
from sklearn.metrics import classification_report
print(classification_report(test_y, Y_text_predictions))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84        96
           1       0.91      0.74      0.82        78
           2       0.99      0.79      0.88        86
           3       0.82      0.81      0.82       104
           4       0.62      0.89      0.73        81

    accuracy                           0.81       445
   macro avg       0.84      0.81      0.82       445
weighted avg       0.84      0.81      0.82       445



In [75]:
story = """
Ex-PM Lord Callaghan dies aged 92

Former Labour prime minister Lord Callaghan has died on the eve of his 93rd birthday.

He passed away at home in East Sussex, just 11 days after his wife Audrey died aged 91. Lord Callaghan, who leaves a son and two daughters, was the longest living former British PM in history. He entered Downing Street in 1976 after the resignation of Harold Wilson. Prime Minister Tony Blair called him a "giant" of the Labour movement. He held each of the major offices of chancellor, home secretary, foreign secretary and prime minister during his career and became Lord Callaghan of Cardiff in 1987.
  
Chancellor Gordon Brown said the former PM would be "mourned throughout the world". "It was a commitment to public service that brought Jim Callaghan into Parliament in 1945, and while Jim rose to the top he never forgot his roots." Former cabinet colleague Lord Hattersley said his first reaction on hearing the news was "immense sadness".

"It was not a major surprise - I knew what a blow the death of his wife Audrey was a few days ago," he said. "He was a decent kindly man who helped me and my generation of politicians immensely.

"The Labour party and the country will be poorer without him." Conservative peer Lord Heseltine said that despite their political differences, he and Lord Callaghan became friends. "You don't get to the premiership unless you have a streak of determination," he said. "But I saw the other side of Jim Callaghan, he became a personal friend in a way, and my family and I were very fond of him." Tory leader Michael Howard said he would be remembered with "affection and respect". Liberal Democrat leader Charles Kennedy said: "When I was first elected in 1983 as the youngest MP, he was the "Father of the House" and as such took a very keen interest in my early days in Parliament.

"He was always full of warmth and wisdom." Born in 1912 and educated at Portsmouth Northern Secondary School, Lord Callaghan became a clerk at the Inland Revenue. He enlisted in the Royal Navy in 1942 and rose to the rank of lieutenant. Elected for a Cardiff constituency at the 1945 general election, he represented Cardiff seats for more than 40 years. After serving as a junior minister in the Attlee government, he became chancellor of the exchequer when Labour returned to power in 1964.
ritish politics into the 21st century.

As foreign secretary in the early 1970s, Lord Callaghan kept an open mind about the UK's entry into the Common Market, seeing the advantages of the UK's entry. He once travelled to Idi Amin's Uganda in 1975 to plead for the life of a British lecturer, Dennis Hills, who was under a death sentence for treason.

His political life was often tempered by battles against the hard left of the party. In the autumn of 1978, before the "Winter of Discontent" when trade unions carried out strikes that brought the country to a standstill, Lord Callaghan refused to hold an early election which may have delivered a Labour victory. As garbage lay uncollected in the streets and hospital staff, council workers and even gravediggers stayed off work, Lord Callaghan failed to predict the mood of the country. When Britain went to the polls in 1979, Tory leader Margaret Thatcher was swept into power in a landslide victory, and Lord Callaghan resigned as Labour leader.
"""

analysed = nlp(story)
genre = le.inverse_transform([svm_clf.predict([list(vectorizer.transform([story]).toarray()[0]) + list(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0])])[0]])[0]

display(genre)

'politics'